In [36]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col,lit,call_udf,countDistinct,sproc,udf
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
import pandas as pd
from configs.config import snowflake_conn_prop_local as snowflake_conn_prop
import sys
import json
import platform
import os,requests
from pathlib import Path
import glob
from src.DataValidationContext import *
from snowflake.snowpark import version

session = Session.builder.configs(snowflake_conn_prop).create()

print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

[Row(CURRENT_WAREHOUSE()='COMPUTE_WH', CURRENT_DATABASE()='TEST', CURRENT_SCHEMA()='VALIDATION')]


In [37]:
import os
import requests
from pathlib import Path
import shutil
import tarfile

# Define project directories
PROJECT_HOME_DIR = '.'
LOCAL_TEMP_DIR = os.path.join(PROJECT_HOME_DIR, 'temp') 
LOCAL_LIB_DIR = os.path.join(LOCAL_TEMP_DIR, 'libs')
LOCAL_TARFile_DIR = os.path.join(LOCAL_TEMP_DIR, 'tarfiles')

# List of library URLs to download
LIB_URLS = [
    'https://files.pythonhosted.org/packages/8e/9d/cecb12289f7967b15facf550a0bbb9c1e910968c3a61b91fd8cdb80aeb3c/great_expectations-0.15.14.tar.gz'
]

for lib_url in LIB_URLS:
    try:
        # Get the file name from the URL
        target_file = lib_url.split('/')[-1]
        local_lib_fl = os.path.join(LOCAL_TARFile_DIR, target_file)
        
        # Create directories if they don't exist
        Path(LOCAL_TARFile_DIR).mkdir(parents=True, exist_ok=True)
        Path(LOCAL_LIB_DIR).mkdir(parents=True, exist_ok=True)
        
        print(f'Downloading library from PyPI to {LOCAL_TARFile_DIR} ...')
        with open(local_lib_fl, "wb") as f:
            r = requests.get(lib_url, timeout=60)
            r.raise_for_status()  # Check if the request was successful
            f.write(r.content)
        print(f'Download complete: {local_lib_fl}')
        
        # Function to extract specific files
        def extract_specific_files(tar_file, target_dir, file_list=None):
            with tarfile.open(tar_file, 'r:gz') as tar:
                if file_list:
                    for member in tar.getmembers():
                        if any(member.name.endswith(file) for file in file_list):
                            tar.extract(member, target_dir)
                            print(f'Extracted {member.name} to {target_dir}')
                else:
                    tar.extractall(target_dir)
                    print(f'Extracted all files to {target_dir}')
        
        # List of specific files you want to extract
        files_to_extract = ['great_expectations/render/notebook_assets/suite_edit/AUTHORING_INTRO.md']
        
        print(f'Started extracting GE tar file to {LOCAL_LIB_DIR}/ge ...')
        extract_specific_files(local_lib_fl, f'{LOCAL_LIB_DIR}/ge', files_to_extract)
        print(f'Done extracting GE tar file to {LOCAL_LIB_DIR}/ge')
    
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the file: {e}")
    except tarfile.TarError as e:
        print(f"Error extracting the tar file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Download complete: ./temp/tarfiles/great_expectations-0.15.14.tar.gz
Started extracting GE tar file to ./temp/libs/ge ...
Extracted great_expectations-0.15.14/great_expectations/render/notebook_assets/suite_edit/AUTHORING_INTRO.md to ./temp/libs/ge
Done extracting GE tar file to ./temp/libs/ge


In [38]:
# Getting the path for the great_expectation folder after the tar file is extracted.
import glob
ge_import_path=''
for result in glob.iglob('./temp/libs/ge/great_expectations*'):
    ge_import_path=result+'/great_expectations'
print(ge_import_path)

./temp/libs/ge/great_expectations-0.15.14/great_expectations


In [41]:
from configs.config import snowflake_conn_prop_local as snowflake_conn_prop
from src.DataValidationContext import GEDataValidationContext
from src.BatchRequest import getBatchRequest
from src.Expectations import  createExpectationSuite, createExpectations
from src.RunLoadExpectations import runExpectaionValidation,loadValidationToDB
from src.parsing import process_and_store_validation_results
from dotenv import load_dotenv
load_dotenv()

import json
import os


session.sql("create or replace stage phani_greatexpectation").collect()
session.clear_packages()
session.add_packages('great-expectations','packaging', 'python-dotenv', 'pyparsing', 'pandas', 'pycryptodomex', 'boto3', 'tzlocal', 'tqdm', 'requests', 'ruamel.yaml', 'ipython', 'jsonpatch', 'mistune', 'jinja2', 'jsonschema', 'scipy', 'altair', 'Click', 'colorama', 'cryptography', 'snowflake-snowpark-python', 'sqlalchemy', 'chardet', 'asn1crypto')
session.clear_imports()
# session.add_import(ge_import_path)
# session.add_import('src')
# session.add_import('configs')


@sproc(session=session,name="GEValidationResults", replace=True, 
       return_type=StringType(),
       input_types=[StringType(),StringType(),StringType()],
       is_permanent=True, stage_location='@phani_greatexpectation/ge_AllLibs')

def generateGEValidationResults(session: Session, db_name: str, schema_name: str ,table_name: str) -> str:
        
    from pathlib import Path
    import os ,sys ,json ,tarfile, dotenv
    
    expecationsuitename = "expecationsuitename"
    checkpointname = "checkpointname"
    
    sftablename = f'{db_name}.VALIDATION.{schema_name}_{table_name}_VALIDATION_RAW'
    destination_table  = f'{db_name}.VALIDATION.{schema_name}_{table_name}_VALIDATION_CLEAN'

    #Creating GE context inside code
    ge=GEDataValidationContext(table_name)
    context=ge.getContext()

    # convert all the date type columns into the same 

    desc_query = f"DESC TABLE {db_name}.{schema_name}.{table_name}"
    columns_info = session.sql(desc_query).collect()
    column_types = {row['name']: row['type'] for row in columns_info}
    
    select_clause = []
    for col, col_type in column_types.items():
        if 'DATE' in col_type or 'TIMESTAMP' in col_type:
            select_clause.append(f"TO_CHAR({col}, 'YYYY-MM-DD') AS {col}")
        else:
            select_clause.append(col)
    query = f"SELECT {', '.join(select_clause)} FROM {db_name}.{schema_name}.{table_name}"

   # Execute the query and convert the results to a pandas DataFrame
    pd_df = session.sql(query).to_pandas()


    # Getting the batch request used while creating and running validation on expectations
    local_batch_request=getBatchRequest(context,table_name,pd_df)
    
    #Creating GE expectation Suite
    createExpectationSuite(context,expecationsuitename)

    #Creating GE expecations
    createExpectations(session, context,expecationsuitename,local_batch_request,pd_df, db_name,schema_name,table_name)

    #Running GE validation 
    res=runExpectaionValidation(context,checkpointname,local_batch_request,expecationsuitename,table_name)
    
    #Loading validation result to Snowflake table.
    loadValidationToDB(session,res,sftablename)


    #Store the parsed data in a clean table
    process_and_store_validation_results(session, sftablename, destination_table)

    return f'VALIDATION RESULTS STORED IN {table_name}_VALIDATION_RAW & CLEAN DATA IN {table_name}_VALIDATION_CLEAN'.upper()

The version of package 'great-expectations' in the local environment is 0.18.5, which does not fit the criteria for the requirement 'great-expectations'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'python-dotenv' in the local environment is 1.0.0, which does not fit the criteria for the requirement 'python-dotenv'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'pyparsing' in the local environment is 3.1.1, which does not fit the criteria for the requirement 'pyparsing'. Your UDF might not work when the package version is different between the server and your local environment.
Package 'pycryptodomex' is not installed in the local environment. Your UDF might not work when the package is installed on the server but not on your local environment.
Package 'boto3' is not installed in the local environment. Your UDF mig

ValueError: stage_location must be specified for permanent stored proc

In [ ]:
#Calling SP
session.sql("call GEValidationResults('DATALAKE','GUS','USERS')").collect()